In [2]:
!pip install sentence_transformers

In [3]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from numpy.linalg import norm

In [99]:
from google.colab import drive

# Monter Google Drive
drive.mount("/content/drive", force_remount=True)
!ls '/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/'
path = "/content/drive/My Drive/Colab Notebooks/Jedha/Projet/data/data_with_best_model_sentiment_analysis.csv"

Mounted at /content/drive
data_avec_labels.csv			     model_roberta_results.csv.csv
data_with_best_model_sentiment_analysis.csv  roberta_results.csv
model_roberta_results.csv


In [4]:
path = "data/data_with_best_model_sentiment_analysis.csv"

In [5]:
df = pd.read_csv(path, encoding='latin-1')

In [6]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'City', 'State',
       'longitude', 'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating', 'actual_sentiment', 'Roberta_label',
       'Roberta_score'],
      dtype='object')

In [7]:
df.head()

,Unnamed: 0,reviewer_id,store_address,City,State,longitude,latitude,review_date,review,clean_reviews,tokenized_reviews,rating,actual_sentiment,Roberta_label,Roberta_score
0,0,1,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-01-15,Why does it look like someone spit on my food?...,why does it look like someone spit on my food ...,look like someone spit food normal transaction...,1,negative,negative,0.975279
1,1,2,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,It'd McDonalds. It is what it is as far as the...,itd mcdonalds it is what it is as far as the f...,itd mcdonalds far food atmosphere go staff mak...,4,positive,positive,0.973995
2,2,3,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,Made a mobile order got to the speaker and che...,made a mobile order got to the speaker and che...,made mobile order got speaker checked line mov...,1,negative,negative,0.775206
3,3,4,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-03-15,My mc. Crispy chicken sandwich was ..............,my mc crispy chicken sandwich was customer ser...,mc crispy chicken sandwich customer service qu...,5,positive,positive,0.901184
4,4,5,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-02-15,"I repeat my order 3 times in the drive thru, a...",i repeat my order times in the drive thru and ...,repeat order times drive thru still manage mes...,1,negative,negative,0.937037


In [8]:
#pour vérifier que la limite de token du model est suffisante
print("longeure moyenne :" , df['review'].apply(len).mean())
print("longeure mediane :" , df['review'].apply(len).median())
print("longeure max :" , df['review'].apply(len).max())
print("longeure min :" , df['review'].apply(len).min())

longeure moyenne : 143.1391011235955
longeure mediane : 82.0
longeure max : 3114
longeure min : 1


# Embedding des labels

In [9]:
first_liste_from_llm = ["Customer Service", "Service", "service","service_experience","Service Quality","Service Refusal",
 "Staff Behavior", "Staffing","Staff Attitude","Manager_Interaction","Staff Friendliness","Staff_Behavior","Management","Behavior",
 "Food Quality", "Food_Quality","food_quality",
 "Operating Hours", "Closing Time",
  "Order Accuracy","Order_Accuracy","Order Resolution","Order","Order Issue","wrong_order",
 "Location", "Location Size","Decor",
 "Dining Experience","General Experience","general_experience","satisfaction", "Overall Experience","overall_experience","experience",
 "Cleanliness","cleanliness",
 "Refund_Policy",
 "Recommendation",
 "Drive-thru experience","drive_through_experience",
 "Safety Concerns","Security",
 "Advice",
 "Receipt Issue",
 "Menu Options","Dissatisfaction with menu","Menu_Options",
 "Customer Loyalty","app_rewards",
 "Product Consistency",
 "Language Barrier",
 "Long Wait Time","Wait_Time","Service Speed","wait_time","service_speed","Speed","slow_service",
 "Left Without Ordering",
 "Product Availability",
 "convenience",
 "Affordability",
"Busyness",
 "Request",
 "Atmosphere",
 "Rating",
 ]


In [10]:
labels=[
  'customer service', 'service', 'service_experience', 'service quality', 'service refusal',
  'staff behavior', 'staffing', 'staff attitude', 'manager_interaction', 'staff friendliness', 'staff_behavior', 'management', 'behavior',
  'food quality', 'food_quality', 'product consistency',
  'operating hours', 'closing time',
  'order accuracy', 'order_accuracy', 'order resolution', 'order', 'order issue', 'wrong_order',
  'location', 'location size', 'decor','atmosphere',
  'dining experience', 'general experience', 'general_experience', 'satisfaction', 'overall experience', 'overall_experience', 'experience',
  'cleanliness',
  'refund_policy',
  'recommendation',
  'drive-thru experience', 'drive_through_experience',
  'safety concerns', 'security',
  'advice',
  'receipt issue',
  'menu options', 'dissatisfaction with menu', 'menu_options',
  'customer loyalty', 'app_rewards',
  'language barrier',
  'long wait time', 'wait_time', 'service speed', 'service_speed', 'speed', 'slow_service',
  'left without ordering',
  'product availability',
  'convenience',
  'affordability',
  'busyness',
  'request',
 ]


In [11]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
labels_embedded = model.encode(labels)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Myriam\anaconda3\envs\myenv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Myriam\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
# Group definitions
label_groups = {
    "customer service": ['customer service', 'service', 'service_experience', 'service quality', 'service refusal'],
    "staff": ['staff behavior', 'staffing', 'staff attitude', 'manager_interaction', 'staff friendliness', 'staff_behavior', 'management', 'behavior'],
    "food quality": ['food quality', 'food_quality', 'product consistency'],
    "operating hours": ['operating hours', 'closing time'],
    "order accuracy": ['order accuracy', 'order_accuracy', 'order resolution', 'order', 'order issue', 'wrong_order'],
    "location": ['location', 'location size', 'decor', 'atmosphere'],
    "general experience": ['dining experience', 'general experience', 'general_experience', 'satisfaction', 'overall experience', 'overall_experience', 'experience'],
    "cleanliness": ['cleanliness'],
    "refund_policy": ['refund_policy'],
    "recommendation": ['recommendation'],
    "drive-thru experience": ['drive-thru experience', 'drive_through_experience'],
    "security": ['safety concerns', 'security'],
    "advice": ['advice'],
    "receipt issue": ['receipt issue'],
    "menu": ['menu options', 'dissatisfaction with menu', 'menu_options'],
    "customer loyalty": ['customer loyalty', 'app_rewards'],
    "language barrier": ['language barrier'],
    "waiting time": ['long wait time', 'wait_time', 'service speed', 'service_speed', 'speed', 'slow_service'],
    "left without ordering": ['left without ordering'],
    "product availability": ['product availability'],
    "convenience": ['convenience'],
    "affordability": ['affordability'],
    "busyness": ['busyness'],
    "request": ['request'],
}

# Inverse mapping: label -> group
label_to_group = {}
for group, group_labels in label_groups.items():
    for label in group_labels:
        label_to_group[label] = group


In [13]:
len(label_groups)

24

# test du l'embedin et dot product sur un seul restaurant

In [14]:
df["store_address"][0]

'13749 US-183 Hwy'

In [15]:
#selection du premier restaurant
df_1restaurant = df[df["store_address"]=="13749 US-183 Hwy"]
df_1restaurant.shape

(583, 15)

In [16]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2') #embeding à 384 dimentions, max token = 256
reviews_embedded = model.encode(df_1restaurant["tokenized_reviews"])
df_1restaurant['review_embedded'] = reviews_embedded.tolist()

C:\Users\Myriam\AppData\Local\Temp\ipykernel_9864\2751898637.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant['review_embedded'] = reviews_embedded.tolist()


In [17]:
df_1restaurant["review_embedded"].shape

(583,)

In [18]:
df_1restaurant['review_embedded'].head()

0    [-0.008658831007778645, -0.06432923674583435, ...
1    [-0.0052186050452291965, -0.0366365946829319, ...
2    [-0.07434768229722977, 0.02320007234811783, 0....
3    [-0.07665494829416275, -0.05378516763448715, 0...
4    [0.05535503849387169, -0.06459932029247284, 0....
Name: review_embedded, dtype: object

In [19]:
from collections import defaultdict
import numpy as np
from numpy.linalg import norm

# Prepare list of list of tuples: top 5 (group, score) per review
top_group_scores = []

for review in reviews_embedded:
    group_scores = defaultdict(list)

    for i, label in enumerate(labels):
        group = label_to_group.get(label, label)
        score = np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i]))
        group_scores[group].append(score)

    # For each group, keep the highest score
    group_max_scores = [(group, max(scores)) for group, scores in group_scores.items()]
    group_max_scores.sort(key=lambda x: x[1], reverse=True)

    # Only top 5 tuples
    top_group_scores.append(group_max_scores[:5])


In [20]:
# Assign each top tuple to its own column
for i in range(5):
    df_1restaurant[f"top_{i+1}"] = [
        group_scores[i] if i < len(group_scores) else (None, None)
        for group_scores in top_group_scores
    ]


C:\Users\Myriam\AppData\Local\Temp\ipykernel_9864\1034889075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f"top_{i+1}"] = [
C:\Users\Myriam\AppData\Local\Temp\ipykernel_9864\1034889075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1restaurant[f"top_{i+1}"] = [
C:\Users\Myriam\AppData\Local\Temp\ipykernel_9864\1034889075.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [21]:
df_1restaurant.head()

,Unnamed: 0,reviewer_id,store_address,City,State,longitude,latitude,review_date,review,clean_reviews,...,rating,actual_sentiment,Roberta_label,Roberta_score,review_embedded,top_1,top_2,top_3,top_4,top_5
0,0,1,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-01-15,Why does it look like someone spit on my food?...,why does it look like someone spit on my food ...,...,1,negative,negative,0.975279,"[-0.008658831007778645, -0.06432923674583435, ...","(food quality, 0.31562415)","(cleanliness, 0.27512175)","(general experience, 0.23939148)","(staff, 0.23229383)","(busyness, 0.19266687)"
1,1,2,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,It'd McDonalds. It is what it is as far as the...,itd mcdonalds it is what it is as far as the f...,...,4,positive,positive,0.973995,"[-0.0052186050452291965, -0.0366365946829319, ...","(general experience, 0.50233704)","(food quality, 0.4219886)","(staff, 0.4207175)","(customer service, 0.35618725)","(customer loyalty, 0.3340216)"
2,2,3,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,Made a mobile order got to the speaker and che...,made a mobile order got to the speaker and che...,...,1,negative,negative,0.775206,"[-0.07434768229722977, 0.02320007234811783, 0....","(customer service, 0.4463125)","(order accuracy, 0.43302265)","(refund_policy, 0.3828023)","(receipt issue, 0.3628762)","(staff, 0.2736548)"
3,3,4,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-03-15,My mc. Crispy chicken sandwich was ..............,my mc crispy chicken sandwich was customer ser...,...,5,positive,positive,0.901184,"[-0.07665494829416275, -0.05378516763448715, 0...","(customer service, 0.39361772)","(food quality, 0.39207894)","(customer loyalty, 0.29757112)","(general experience, 0.2721585)","(order accuracy, 0.26110744)"
4,4,5,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-02-15,"I repeat my order 3 times in the drive thru, a...",i repeat my order times in the drive thru and ...,...,1,negative,negative,0.937037,"[0.05535503849387169, -0.06459932029247284, 0....","(order accuracy, 0.4382611)","(drive-thru experience, 0.41875172)","(general experience, 0.33675212)","(food quality, 0.29102874)","(busyness, 0.2879883)"


In [22]:
# List of all unique group labels
all_group_labels = list(set(label_to_group.values()))

# This will be a list of dicts, one per review
group_score_dicts = []

for review in reviews_embedded:
    group_scores = defaultdict(list)

    for i, label in enumerate(labels):
        group = label_to_group.get(label, label)
        score = np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i]))
        group_scores[group].append(score)

    # Take max score per group
    group_max_scores = {group: max(scores) for group, scores in group_scores.items()}
    group_score_dicts.append(group_max_scores)


In [23]:
# Convert list of dicts to DataFrame (each group becomes a column)
group_scores_df = pd.DataFrame(group_score_dicts)

# Merge into your main DataFrame
df_1restaurant = pd.concat([df_1restaurant.reset_index(drop=True), group_scores_df], axis=1)


In [24]:
df_1restaurant.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'City', 'State',
       'longitude', 'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating', 'actual_sentiment', 'Roberta_label',
       'Roberta_score', 'review_embedded', 'top_1', 'top_2', 'top_3', 'top_4',
       'top_5', 'customer service', 'staff', 'food quality', 'operating hours',
       'order accuracy', 'location', 'general experience', 'cleanliness',
       'refund_policy', 'recommendation', 'drive-thru experience', 'security',
       'advice', 'receipt issue', 'menu', 'customer loyalty',
       'language barrier', 'waiting time', 'left without ordering',
       'product availability', 'convenience', 'affordability', 'busyness',
       'request'],
      dtype='object')

In [25]:
df_1restaurant= df_1restaurant[['reviewer_id', 'store_address', 'City', 'State',
       'longitude', 'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating',
       'actual_sentiment', 'Roberta_label', 'Roberta_score',
       'top_1', 'top_2', 'top_3', 'top_4', 'top_5', 'customer service',
       'staff', 'food quality', 'operating hours', 'order accuracy',
       'location', 'general experience', 'cleanliness', 'refund_policy',
       'recommendation', 'drive-thru experience', 'security', 'advice',
       'receipt issue', 'menu', 'customer loyalty', 'language barrier',
       'waiting time', 'left without ordering', 'product availability',
       'convenience', 'affordability', 'busyness', 'request']]

In [26]:
df_1restaurant[['review',
       'actual_sentiment', 'Roberta_label',
       'top_1', 'top_2', 'top_3', 'top_4', 'top_5', 'customer service',
       'staff', 'food quality', 'operating hours', 'order accuracy',
       'location', 'general experience', 'cleanliness', 'refund_policy',
       'recommendation', 'drive-thru experience', 'security', 'advice',
       'receipt issue', 'menu', 'customer loyalty', 'language barrier',
       'waiting time', 'left without ordering', 'product availability',
       'convenience', 'affordability', 'busyness', 'request']].head()

,review,actual_sentiment,Roberta_label,top_1,top_2,top_3,top_4,top_5,customer service,staff,...,menu,customer loyalty,language barrier,waiting time,left without ordering,product availability,convenience,affordability,busyness,request
0,Why does it look like someone spit on my food?...,negative,negative,"(food quality, 0.31562415)","(cleanliness, 0.27512175)","(general experience, 0.23939148)","(staff, 0.23229383)","(busyness, 0.19266687)",0.155685,0.232294,...,0.158993,0.074503,0.112069,0.141913,0.139210,0.112632,0.095136,0.010976,0.192667,0.132173
1,It'd McDonalds. It is what it is as far as the...,positive,positive,"(general experience, 0.50233704)","(food quality, 0.4219886)","(staff, 0.4207175)","(customer service, 0.35618725)","(customer loyalty, 0.3340216)",0.356187,0.420718,...,0.233368,0.334022,0.090067,0.185619,0.071344,0.103034,0.268924,0.173768,0.214261,-0.001474
2,Made a mobile order got to the speaker and che...,negative,negative,"(customer service, 0.4463125)","(order accuracy, 0.43302265)","(refund_policy, 0.3828023)","(receipt issue, 0.3628762)","(staff, 0.2736548)",0.446312,0.273655,...,0.194078,0.263592,0.146208,0.235918,0.259681,0.184906,0.142167,0.091285,0.226593,0.101869
3,My mc. Crispy chicken sandwich was ..............,positive,positive,"(customer service, 0.39361772)","(food quality, 0.39207894)","(customer loyalty, 0.29757112)","(general experience, 0.2721585)","(order accuracy, 0.26110744)",0.393618,0.216679,...,0.202433,0.297571,-0.000655,0.221744,0.139198,0.162949,0.133394,0.095757,0.143017,0.158758
4,"I repeat my order 3 times in the drive thru, a...",negative,negative,"(order accuracy, 0.4382611)","(drive-thru experience, 0.41875172)","(general experience, 0.33675212)","(food quality, 0.29102874)","(busyness, 0.2879883)",0.258598,0.262975,...,0.277522,0.119351,0.226186,0.234919,0.234002,0.115486,0.218603,0.180055,0.287988,0.026835


In [27]:
'''
labels = [
    'hygiene', 'food quality', 'food', 'staff', 'something is missing',
    'location', 'speed of service', 'drive-thru', 'temperature of the food',
    'atmosphere', 'customer service', "temperature" , "price", "speed", "quality", "courtesy",
]
'''

'\nlabels = [\n    \'hygiene\', \'food quality\', \'food\', \'staff\', \'something is missing\',\n    \'location\', \'speed of service\', \'drive-thru\', \'temperature of the food\',\n    \'atmosphere\', \'customer service\', "temperature" , "price", "speed", "quality", "courtesy",\n]\n'

In [28]:
df_1restaurant.columns

Index(['reviewer_id', 'store_address', 'City', 'State', 'longitude',
       'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating', 'actual_sentiment', 'Roberta_label',
       'Roberta_score', 'top_1', 'top_2', 'top_3', 'top_4', 'top_5',
       'customer service', 'staff', 'food quality', 'operating hours',
       'order accuracy', 'location', 'general experience', 'cleanliness',
       'refund_policy', 'recommendation', 'drive-thru experience', 'security',
       'advice', 'receipt issue', 'menu', 'customer loyalty',
       'language barrier', 'waiting time', 'left without ordering',
       'product availability', 'convenience', 'affordability', 'busyness',
       'request'],
      dtype='object')

# Embedding et dot product sur tout le dataset

In [29]:
from tqdm.notebook import tqdm

# Remplacer les valeurs manquantes par une chaîne vide
df['tokenized_reviews'] = df['tokenized_reviews'].fillna('')

# Charger le modèle SentenceTransformer
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Encoder les avis avec une barre de progression
reviews_embedded = []
for review in tqdm(df["tokenized_reviews"], desc="Encoding reviews"):
    reviews_embedded.append(model.encode(review))

# Ajouter les embeddings au DataFrame
df['review_embedded'] = reviews_embedded

Encoding reviews:   0%|          | 0/22250 [00:00<?, ?it/s]

In [30]:
df.to_csv("data/data_with_topics.csv", index=False)
df= pd.read_csv("data/data_with_topics.csv", encoding='latin-1')

In [31]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'City', 'State',
       'longitude', 'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating', 'actual_sentiment', 'Roberta_label',
       'Roberta_score', 'review_embedded'],
      dtype='object')

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22250 entries, 0 to 22249
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         22250 non-null  int64  
 1   reviewer_id        22250 non-null  int64  
 2   store_address      22250 non-null  object 
 3   City               22250 non-null  object 
 4   State              22250 non-null  object 
 5   longitude          22250 non-null  float64
 6   latitude           22250 non-null  float64
 7   review_date        22250 non-null  object 
 8   review             22250 non-null  object 
 9   clean_reviews      22250 non-null  object 
 10  tokenized_reviews  22179 non-null  object 
 11  rating             22250 non-null  int64  
 12  actual_sentiment   22250 non-null  object 
 13  Roberta_label      22250 non-null  object 
 14  Roberta_score      22250 non-null  float64
 15  review_embedded    22250 non-null  object 
dtypes: float64(3), int64(3

In [33]:
from collections import defaultdict
import numpy as np
from numpy.linalg import norm

# Prepare list of list of tuples: top 5 (group, score) per review
top_group_scores = []

for review in reviews_embedded:
    group_scores = defaultdict(list)

    for i, label in enumerate(labels):
        group = label_to_group.get(label, label)
        score = np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i]))
        group_scores[group].append(score)

    # For each group, keep the highest score
    group_max_scores = [(group, max(scores)) for group, scores in group_scores.items()]
    group_max_scores.sort(key=lambda x: x[1], reverse=True)

    # Only top 5 tuples
    top_group_scores.append(group_max_scores[:5])

# Assign each top tuple to its own column
for i in range(5):
    df[f"top_{i+1}"] = [
        group_scores[i] if i < len(group_scores) else (None, None)
        for group_scores in top_group_scores
    ]



In [34]:
df.head()

,Unnamed: 0,reviewer_id,store_address,City,State,longitude,latitude,review_date,review,clean_reviews,...,rating,actual_sentiment,Roberta_label,Roberta_score,review_embedded,top_1,top_2,top_3,top_4,top_5
0,0,1,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-01-15,Why does it look like someone spit on my food?...,why does it look like someone spit on my food ...,...,1,negative,negative,0.975279,[-8.65883566e-03 -6.43292069e-02 6.25042990e-...,"(food quality, 0.31562406)","(cleanliness, 0.27512178)","(general experience, 0.23939143)","(staff, 0.23229381)","(busyness, 0.19266683)"
1,1,2,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,It'd McDonalds. It is what it is as far as the...,itd mcdonalds it is what it is as far as the f...,...,4,positive,positive,0.973995,[-5.21856779e-03 -3.66366133e-02 9.62128639e-...,"(general experience, 0.5023371)","(food quality, 0.4219886)","(staff, 0.42071745)","(customer service, 0.35618728)","(customer loyalty, 0.33402154)"
2,2,3,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,Made a mobile order got to the speaker and che...,made a mobile order got to the speaker and che...,...,1,negative,negative,0.775206,[-7.43476525e-02 2.32000630e-02 1.82521679e-...,"(customer service, 0.44631246)","(order accuracy, 0.43302262)","(refund_policy, 0.38280222)","(receipt issue, 0.3628762)","(staff, 0.27365482)"
3,3,4,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-03-15,My mc. Crispy chicken sandwich was ..............,my mc crispy chicken sandwich was customer ser...,...,5,positive,positive,0.901184,[-7.66549706e-02 -5.37851751e-02 5.62056936e-...,"(customer service, 0.39361775)","(food quality, 0.3920789)","(customer loyalty, 0.29757115)","(general experience, 0.27215844)","(order accuracy, 0.26110753)"
4,4,5,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-02-15,"I repeat my order 3 times in the drive thru, a...",i repeat my order times in the drive thru and ...,...,1,negative,negative,0.937037,[ 5.53550310e-02 -6.45993128e-02 8.45504552e-...,"(order accuracy, 0.43826103)","(drive-thru experience, 0.41875166)","(general experience, 0.3367521)","(food quality, 0.29102868)","(busyness, 0.2879883)"


In [35]:
# List of all unique group labels
all_group_labels = list(set(label_to_group.values()))

# This will be a list of dicts, one per review
group_score_dicts = []

for review in reviews_embedded:
    group_scores = defaultdict(list)

    for i, label in enumerate(labels):
        group = label_to_group.get(label, label)
        score = np.dot(review, labels_embedded[i]) / (norm(review) * norm(labels_embedded[i]))
        group_scores[group].append(score)

    # Take max score per group
    group_max_scores = {group: max(scores) for group, scores in group_scores.items()}
    group_score_dicts.append(group_max_scores)

# Convert list of dicts to DataFrame (each group becomes a column)
group_scores_df = pd.DataFrame(group_score_dicts)

# Merge into your main DataFrame
df = pd.concat([df.reset_index(drop=True), group_scores_df], axis=1)


In [36]:
df[['review',
       'actual_sentiment', 'Roberta_label',
       'top_1', 'top_2', 'top_3', 'top_4', 'top_5', 'customer service',
       'staff', 'food quality', 'operating hours', 'order accuracy',
       'location', 'general experience', 'cleanliness', 'refund_policy',
       'recommendation', 'drive-thru experience', 'security', 'advice',
       'receipt issue', 'menu', 'customer loyalty', 'language barrier',
       'waiting time', 'left without ordering', 'product availability',
       'convenience', 'affordability', 'busyness', 'request']].head()

,review,actual_sentiment,Roberta_label,top_1,top_2,top_3,top_4,top_5,customer service,staff,...,menu,customer loyalty,language barrier,waiting time,left without ordering,product availability,convenience,affordability,busyness,request
0,Why does it look like someone spit on my food?...,negative,negative,"(food quality, 0.31562406)","(cleanliness, 0.27512178)","(general experience, 0.23939143)","(staff, 0.23229381)","(busyness, 0.19266683)",0.155685,0.232294,...,0.158993,0.074503,0.112069,0.141913,0.139210,0.112632,0.095136,0.010976,0.192667,0.132173
1,It'd McDonalds. It is what it is as far as the...,positive,positive,"(general experience, 0.5023371)","(food quality, 0.4219886)","(staff, 0.42071745)","(customer service, 0.35618728)","(customer loyalty, 0.33402154)",0.356187,0.420717,...,0.233368,0.334022,0.090067,0.185619,0.071343,0.103034,0.268924,0.173768,0.214261,-0.001474
2,Made a mobile order got to the speaker and che...,negative,negative,"(customer service, 0.44631246)","(order accuracy, 0.43302262)","(refund_policy, 0.38280222)","(receipt issue, 0.3628762)","(staff, 0.27365482)",0.446312,0.273655,...,0.194078,0.263592,0.146208,0.235918,0.259681,0.184906,0.142167,0.091285,0.226593,0.101869
3,My mc. Crispy chicken sandwich was ..............,positive,positive,"(customer service, 0.39361775)","(food quality, 0.3920789)","(customer loyalty, 0.29757115)","(general experience, 0.27215844)","(order accuracy, 0.26110753)",0.393618,0.216679,...,0.202433,0.297571,-0.000655,0.221744,0.139198,0.162949,0.133394,0.095757,0.143017,0.158758
4,"I repeat my order 3 times in the drive thru, a...",negative,negative,"(order accuracy, 0.43826103)","(drive-thru experience, 0.41875166)","(general experience, 0.3367521)","(food quality, 0.29102868)","(busyness, 0.2879883)",0.258598,0.262975,...,0.277522,0.119351,0.226186,0.234919,0.234002,0.115486,0.218603,0.180055,0.287988,0.026835


In [37]:
df.columns

Index(['Unnamed: 0', 'reviewer_id', 'store_address', 'City', 'State',
       'longitude', 'latitude', 'review_date', 'review', 'clean_reviews',
       'tokenized_reviews', 'rating', 'actual_sentiment', 'Roberta_label',
       'Roberta_score', 'review_embedded', 'top_1', 'top_2', 'top_3', 'top_4',
       'top_5', 'customer service', 'staff', 'food quality', 'operating hours',
       'order accuracy', 'location', 'general experience', 'cleanliness',
       'refund_policy', 'recommendation', 'drive-thru experience', 'security',
       'advice', 'receipt issue', 'menu', 'customer loyalty',
       'language barrier', 'waiting time', 'left without ordering',
       'product availability', 'convenience', 'affordability', 'busyness',
       'request'],
      dtype='object')

In [38]:
df=df[['reviewer_id',
       'store_address', 'City', 'State','longitude', 'latitude',
       'review_date', 'review', 'clean_reviews','tokenized_reviews',
       'rating', 'actual_sentiment', 'Roberta_label','Roberta_score',
       'top_1', 'top_2', 'top_3', 'top_4','top_5',
       'customer service', 'staff', 'food quality', 'operating hours',
       'order accuracy', 'location', 'general experience', 'cleanliness',
       'refund_policy', 'recommendation', 'drive-thru experience', 'security',
       'advice', 'receipt issue', 'menu', 'customer loyalty',
       'language barrier', 'waiting time', 'left without ordering',
       'product availability', 'convenience', 'affordability', 'busyness',
       'request' ]]
df.head()

,reviewer_id,store_address,City,State,longitude,latitude,review_date,review,clean_reviews,tokenized_reviews,...,menu,customer loyalty,language barrier,waiting time,left without ordering,product availability,convenience,affordability,busyness,request
0,1,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-01-15,Why does it look like someone spit on my food?...,why does it look like someone spit on my food ...,look like someone spit food normal transaction...,...,0.158993,0.074503,0.112069,0.141913,0.139210,0.112632,0.095136,0.010976,0.192667,0.132173
1,2,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,It'd McDonalds. It is what it is as far as the...,itd mcdonalds it is what it is as far as the f...,itd mcdonalds far food atmosphere go staff mak...,...,0.233368,0.334022,0.090067,0.185619,0.071343,0.103034,0.268924,0.173768,0.214261,-0.001474
2,3,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-04-10,Made a mobile order got to the speaker and che...,made a mobile order got to the speaker and che...,made mobile order got speaker checked line mov...,...,0.194078,0.263592,0.146208,0.235918,0.259681,0.184906,0.142167,0.091285,0.226593,0.101869
3,4,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-03-15,My mc. Crispy chicken sandwich was ..............,my mc crispy chicken sandwich was customer ser...,mc crispy chicken sandwich customer service qu...,...,0.202433,0.297571,-0.000655,0.221744,0.139198,0.162949,0.133394,0.095757,0.143017,0.158758
4,5,13749 US-183 Hwy,Austin,TX,-97.792874,30.460718,2025-02-15,"I repeat my order 3 times in the drive thru, a...",i repeat my order times in the drive thru and ...,repeat order times drive thru still manage mes...,...,0.277522,0.119351,0.226186,0.234919,0.234002,0.115486,0.218603,0.180055,0.287988,0.026835


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22250 entries, 0 to 22249
Data columns (total 43 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   reviewer_id            22250 non-null  int64  
 1   store_address          22250 non-null  object 
 2   City                   22250 non-null  object 
 3   State                  22250 non-null  object 
 4   longitude              22250 non-null  float64
 5   latitude               22250 non-null  float64
 6   review_date            22250 non-null  object 
 7   review                 22250 non-null  object 
 8   clean_reviews          22250 non-null  object 
 9   tokenized_reviews      22179 non-null  object 
 10  rating                 22250 non-null  int64  
 11  actual_sentiment       22250 non-null  object 
 12  Roberta_label          22250 non-null  object 
 13  Roberta_score          22250 non-null  float64
 14  top_1                  22250 non-null  object 
 15  to

In [40]:
df.shape

(22250, 43)

In [41]:
df.to_csv("data/data_with_topics.csv", index=False)

In [42]:
# Convertir les colonnes de labels en numériques
for label in df.columns[19:43]:
    df[label] = pd.to_numeric(df[label], errors='coerce')

In [43]:
df.to_csv("data/data_with_topics.csv", index=False)

In [44]:
contains_str = df.applymap(lambda x: isinstance(x, str)).any()
contains_str[contains_str==True]

C:\Users\Myriam\AppData\Local\Temp\ipykernel_9864\4058449201.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  contains_str = df.applymap(lambda x: isinstance(x, str)).any()


store_address        True
City                 True
State                True
review_date          True
review               True
clean_reviews        True
tokenized_reviews    True
actual_sentiment     True
Roberta_label        True
dtype: bool